In [46]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as ml
import matplotlib.pyplot as plt
import datetime
import multiprocessing as mp
import pickle
import statistics
import csv

def parseStage(stage):
    return float(stage[1:])

print('Here')
print(str(datetime.datetime.now()))

Here
2019-05-02 09:20:56.190023


In [2]:
rawGeneToGoidData = pd.read_csv('MOCA-data/endoderm_counts/associations/mgi_annotations.mapped.csv')
rawGoidToNameData = pd.read_csv('MOCA-data/endoderm_counts/associations/go_terms_slim_mouse.csv')
rawSparseMatrix = pd.read_csv('MOCA-data/endoderm_counts/endoderm_counts.dat', delimiter=' ')
rawMetaData = pd.read_csv('MOCA-data/endoderm_counts/endoderm_meta.csv')
rawGeneList = pd.read_csv('MOCA-data/endoderm_counts/genesList.csv')

print('Here')
print(str(datetime.datetime.now()))

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Here
2019-05-01 16:23:25.270539


In [3]:
#properly indexed genes
genes = [element.upper() for element in list(rawGeneList.Symbol)]

print('Here')
print(str(datetime.datetime.now()))

Here
2019-05-01 16:23:25.278828


In [4]:
#map of gene name to unique set of GO_ID
geneToGoidMap = {}

for val in list(zip(rawGeneToGoidData.DB_object_symbol, rawGeneToGoidData.GO_ID)):
    gene = val[0].upper()
    if not gene in geneToGoidMap:
        geneToGoidMap[gene] = set()
        
    geneToGoidMap[gene].add(val[1])

print('Here')
print(str(datetime.datetime.now()))

Here
2019-05-01 16:23:25.529920


In [39]:
#map of GO_ID to variable name
goidToVariableNameMap = {}
goidToAspectMap = {}

for val in list(zip(rawGoidToNameData.GO_ID, rawGoidToNameData.Name, rawGoidToNameData.Aspect)):
    assert not(val[0] in goidToVariableNameMap), "Duplicate GOID " + val[0]
    goidToVariableNameMap[val[0]] = val[1]
    goidToAspectMap[val[0]] = val[2]
        
print('Here')
print(str(datetime.datetime.now()))

Here
2019-05-02 09:15:27.161973


In [34]:
print("Starting")
print(str(datetime.datetime.now()))

class RowResult:
    empty = True
    gene = ''
    missing = False
    found = False
    pickledValuesByGoidByTime = None

def processRow(row):
    ret = RowResult()
    
    if row[0] % 1000000 == 0:
        print('Processing Sample: ' + str(row[0]))
    
    if row[1] - 1 >= len(genes):
        print("Gene Index Error: " + str(row))
        return ret
        
    gene = genes[row[1]- 1]
    ret.gene = gene
    ret.empty = False
    
    if not gene in geneToGoidMap:
        ret.missing = True
        return ret
    else:
        ret.found = True
        
    if row[2] - 1 >= len(rawMetaData.index):
        print('Meta Index Error: ' + str(row))
        return ret;
    
    stage = rawMetaData.iloc[row[2] - 1].stage
    if stage == 'mixed_gastrulation':
        return ret
    
    time = parseStage(stage)
        
    goids = geneToGoidMap[gene]
    valuesByGoidByTime = {}
    for goid in goids:
        if not goid in valuesByGoidByTime:
            valuesByGoidByTime[goid] = {}
            
        if not time in valuesByGoidByTime[goid]:
            valuesByGoidByTime[goid][time] = []
        
        valuesByGoidByTime[goid][time].append(row[3])
    
    ret.pickledValuesByGoidByTime = pickle.dumps(valuesByGoidByTime)
    return ret

#map of goid to map of time to list of value
valuesByGoidByTime = {}
#these are the genes that don't fit our aggregation model (AKA not in goslim)
missingGenes = set()
#these are the genes that do fit our aggregation model
foundGenes = set()

with mp.Pool(mp.cpu_count()) as pool:
    result = pool.imap(processRow, rawSparseMatrix.itertuples(name=None), chunksize=100000)
    output = [x for x in result]
    for ret in output:
        if ret.empty:
            continue;
        
        if ret.missing:
            missingGenes.add(ret.gene)
        elif ret.found:
            foundGenes.add(ret.gene)
            if ret.pickledValuesByGoidByTime: #mixed_gastrulation will be found, but null
                for goid, retValuesByGoidByTime in pickle.loads(ret.pickledValuesByGoidByTime).items():
                    if not goid in valuesByGoidByTime:
                        valuesByGoidByTime[goid] = {}

                    for time, values in retValuesByGoidByTime.items():
                        if not time in valuesByGoidByTime[goid]:
                            valuesByGoidByTime[goid][time] = []

                        valuesByGoidByTime[goid][time].extend(values);
                        
print('Genes which arent in goslim ' + str(len(missingGenes)))
print('Genes which are in goslim ' + str(len(foundGenes)))
print(str(datetime.datetime.now()))
print(len(valuesByGoidByTime.keys()))

Starting
2019-05-01 17:22:58.683876
Processing Sample: 0
Processing Sample: 1000000
Processing Sample: 2000000
Processing Sample: 3000000
Processing Sample: 4000000
Processing Sample: 5000000
Processing Sample: 6000000
Processing Sample: 7000000
Processing Sample: 8000000
Processing Sample: 9000000
Processing Sample: 10000000
Processing Sample: 11000000
Processing Sample: 12000000
Processing Sample: 13000000
Processing Sample: 14000000
Processing Sample: 15000000
Processing Sample: 16000000
Processing Sample: 17000000
Genes which arent in goslim 4550
Genes which are in goslim 14746
2019-05-01 17:36:02.456952
44


In [51]:
stats = ['mean', 'stdev', 'variance'] #these are methods on the python3 statistics module that we want to run

for stat in stats:
    print("Starting " + stat)
    print(str(datetime.datetime.now()))
    csvData = []
    header = ['Aspect', 'Variable']
    timeRange = np.linspace(6.5, 8.5, 9)

    for time in timeRange:
        header.append(str(time))

    csvData.append(header)

    for goid, valuesByTime in valuesByGoidByTime.items():
        row = []
        variableName = goidToVariableNameMap[goid]
        aspect = goidToAspectMap[goid]

        row.append(aspect)
        row.append(variableName)

        for time in timeRange:
            method = getattr(statistics, stat)
            values = valuesByTime[time]
            row.append(method(values))

        csvData.append(row)

    with open('out/aggregation_' + stat + '.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for row in csvData:
            filewriter.writerow(row)

    print('Done -- open out/aggregation_' + stat + '.csv')
    print(str(datetime.datetime.now()))
    
print('Done Done')

Starting mean
2019-05-02 09:30:13.420902
Done -- open out/aggregation_mean.csv
2019-05-02 09:31:06.790248
Starting median
2019-05-02 09:31:06.790365
Done -- open out/aggregation_median.csv
2019-05-02 09:31:13.924520
Starting stdev
2019-05-02 09:31:13.924954
Done -- open out/aggregation_stdev.csv
2019-05-02 09:34:36.316301
Starting variance
2019-05-02 09:34:36.316333
Done -- open out/aggregation_variance.csv
2019-05-02 09:37:56.272871
